In [2]:
import pandas as pd
import numpy as np
import time
import os
import random
#from urbansim.models import transition, relocation
from urbansim.developer import sqftproforma, developer
from urbansim.utils import misc, networks
#import dataset, variables, utils, transcad
import dataset, variables, utils
import pandana as pdna
import models

import orca

In [3]:
orca.run(["build_networks"])
orca.run(["neighborhood_vars"])

Running step 'build_networks'


dataset.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  buildings.sqft_price_nonres[buildings.sqft_price_nonres==np.inf] = 0
dataset.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  buildings.sqft_price_res[buildings.sqft_price_res==np.inf] = 0


Time to execute step 'build_networks': 96.50 s
Total time to execute iteration 1 with iteration value None: 96.50 s
Running step 'neighborhood_vars'


dataset.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.building_id[df.building_id==-1] = np.random.choice(store.buildings.index.values,(df.building_id==-1).sum())
dataset.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df.building_id[idx_invalid_building_id] = np.random.choice(store.buildings.index.values,idx_invalid_building_id.sum())
models.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  j.building_id[idx_invalid_building_id] = np.random.choice(b[np.in1d(b.large_a

Computing accessibility variables
Computing sum_residential_units
Removed 7 rows because they contain missing values
Computing sum_nonresidential_units
Removed 7 rows because they contain missing values
Computing ave_unit_sqft
Removed 1 rows because they contain missing values
Computing ave_lot_sqft
Removed 7 rows because they contain missing values
Computing population
Removed 2 rows because they contain missing values
Computing poor
Removed 1 rows because they contain missing values
Computing sfdu
Computing hhsize
Removed 2 rows because they contain missing values
Computing ave_income
Removed 2 rows because they contain missing values
Computing residential
Removed 2 rows because they contain missing values
Computing retail
Computing office
Removed 1 rows because they contain missing values
Computing industrial
Computing medical
Computing max_industrial_far
Computing max_office_far
Computing max_retail_far
Computing max_medical_far


/home/da/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


       sum_residential_units  sum_nonresidential_units  ave_unit_sqft  \
count          270914.000000             270914.000000  270914.000000   
mean                1.637083                  0.717733    1115.246094   
std                 1.500576                  1.522981    1084.281494   
min                 0.000000                  0.000000       0.000000   
25%                 0.000000                  0.000000       0.000000   
50%                 1.844241                  0.000000    1142.758118   
75%                 2.898982                  0.000000    1811.661896   
max                 7.384610                 10.669583   23051.000000   

        ave_lot_sqft     population           poor      sfdu         hhsize  \
count  270914.000000  270914.000000  270914.000000  270914.0  270914.000000   
mean        5.056724       2.121810       1.164474       0.0       0.785301   
std         3.711901       1.865198       1.401323       0.0       0.636225   
min         0.000000      

## Hedonic Estimation

In [3]:
orca.run(["rsh_estimate"])

Running step 'rsh_estimate'


KeyError: 'nodes'

In [ ]:
orca.run(["nrh_estimate"])

In [4]:
orca.list_tables()

['travel_data',
 'nodes_prices',
 'tazcounts2015gq',
 'zones',
 'large_areas',
 'access_drive_minutes',
 'nodes_drv',
 'tazcounts2035gq',
 'employment_sectors',
 'households',
 'annual_employment_control_totals',
 'annual_relocation_rates_for_households',
 'nodes',
 'building_types',
 'buildings',
 'jobs',
 'tazcounts2030gq',
 'access_walk_feet',
 'tazcounts2040gq',
 'zoning',
 'persons',
 'annual_household_control_totals',
 'cities',
 'annual_relocation_rates_for_jobs',
 'counties',
 'land_use_types',
 'target_vacancies',
 'home_based_status',
 'tazcounts2020gq',
 'nodes_walk',
 'tazcounts2025gq',
 'scheduled_development_events',
 'building_sqft_per_job',
 'parcels']

In [5]:
orca.get_table('nodes_walk').to_frame()

,sum_residential_units,sum_nonresidential_units,ave_unit_sqft,ave_lot_sqft,population,poor,sfdu,hhsize,ave_income,residential,retail,office,industrial,medical,max_industrial_far,max_office_far,max_retail_far,max_medical_far
0,2.690346,0.000000,2633.027100,7.660164,3.753546,1.870682,0.0,1.432507,11.576988,169.132156,141.280487,0.000000,0.000000,0.000000,-1.000000,0.150052,0.253973,-1.000000
1,2.308411,0.924141,1268.947388,7.019222,2.759260,2.039568,0.0,0.998529,10.511455,59.268692,80.717773,0.000000,0.000000,116.503944,1.404835,1.512509,1.611425,0.540413
2,2.082935,0.000000,1699.304321,7.515487,2.713286,1.389352,0.0,1.303407,11.056359,109.818863,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000
4,3.325014,1.060764,1043.016968,6.946163,4.289271,2.641370,0.0,1.282916,10.954058,77.722267,103.342430,145.216110,38.707706,0.000000,0.568492,1.034518,1.136686,0.319644
5,3.282348,0.000000,1323.590942,7.070014,4.115971,2.332292,0.0,1.263162,11.297665,177.484055,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.071515,-1.000000
6,2.197225,0.000000,1892.250000,7.340419,3.135494,1.098612,0.0,1.321756,11.501387,133.969467,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000
7,2.613957,0.000000,1991.439941,7.353184,3.465077,2.459943,0.0,1.376244,11.233729,71.997086,0.000000,0.000000,24.102234,0.000000,0.196986,0.237038,0.900743,0.148184
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,49.377613,107.914955,11.253384,0.000000,116.503944,1.404835,1.512509,1.006646,0.540413
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000


## LCM Estimation

In [4]:
orca.run(["hlcm_estimate"])

Running step 'hlcm_estimate'


KeyError: 'nodes'

In [5]:
orca.run(["elcm_estimate"])

Running step 'elcm_estimate'


KeyError: 'nodes'